# Samping raw data to simulate data used for dev, stag and prod

split the chronological timeline into Dev (oldest) → Staging (recent past) → Prod (most recent). Each environment can then be used differently.

split the 6.36M rows into:
- Dev → ~70% oldest steps
- Staging → ~15% middle steps
- Prod → ~15% newest steps

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
RAW_PATH = Path("../data/raw")
CREDITCARD_PATH = RAW_PATH / "paysim_data.csv"

df = pd.read_csv(CREDITCARD_PATH)
df.head()
len(df)

6362620

To avoid data leakage & keep fraud class balance:
Use stratified sampling on the target column (Class) so the fraud ratio stays the same.

In [ ]:
# Check unique step values (time simulation)
print("Step range:", df["step"].min(), "to", df["step"].max())

# Split by quantiles on step
q1 = df["step"].quantile(0.70)   # 70% cutoff
q2 = df["step"].quantile(0.85)   # 85% cutoff

dev_df = df[df["step"] <= q1].copy()
staging_df = df[(df["step"] > q1) & (df["step"] <= q2)].copy()
prod_df = df[df["step"] > q2].copy()

dev_df.to_csv(RAW_PATH / "paysim-dev.csv", index=False)
staging_df.to_csv(RAW_PATH / "paysim-staging.csv", index=False)
prod_df.to_csv(RAW_PATH / "paysim-prod.csv", index=False)

print(f"Dev: {len(dev_df)} rows, Staging: {len(staging_df)} rows, Prod: {len(prod_df)} rows")
print("Fraud rates:", dev_df['isFraud'].mean(), staging_df['isFraud'].mean(), prod_df['isFraud'].mean())

Step range: 1 to 743
Dev: 4463587 rows, Staging: 980416 rows, Prod: 918617 rows
Fraud rates: 0.0008161597387930379 0.0005752660095306482 0.004360903401526425


In [ ]:
# then push the data to s3 via dvc
eg
dvc add data/raw/paysim-prod.csv
dvc push

SyntaxError: invalid syntax (962431800.py, line 3)